# 04 — Final Prediction & Kaggle Submission

本 Notebook 將訓練好的模型套用在 test dataset，產生 Kaggle 競賽要求格式的 submission.csv

---

## 📌 目標

- 對 test.csv 進行與 train 相同的清洗
- 套用訓練好的模型
- 產出最終預測結果

---

## 📘 目錄

### 1. 匯入 test 資料
- 讀取 test.csv
- 檢查缺失值

### 2. 套用與 train 相同的清洗流程
- 補 Age / Fare
- FamilySize = SibSp + Parch + 1
- Sex 編碼
- Embarked one-hot（確保欄位一致）
- 刪除 Cabin

### 3. 預測 Survived
- model.predict(test_features)
- 建立預測欄位

### 4. 建立 submission.csv
- PassengerId 與預測結果合併
- 輸出至 submission.csv

---

## 🎯 最終成果

成功提交到 Kaggle 的 submission.csv

你可以在作品集中說明：
- Score：0.75119
- 使用模型：Random Forest

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# 讀取 test 與 清洗後的 train
test = pd.read_csv("test.csv")
train_clean = pd.read_csv("train_clean.csv")

test.head()
print(test.head())
print(test.dtypes)

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          

In [2]:
# 1. FamilySize
test["FamilySize"] = test["SibSp"] + test["Parch"] + 1

# 2. Age 缺失值補中位數
test["Age"] = test["Age"].fillna(test["Age"].median())

# 3. Fare 缺失值補中位數
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

# 4. Sex → 0/1
test["Sex"] = test["Sex"].map({"male": 0, "female": 1})

# 5. Embarked one-hot（要和 train 一模一樣）
test["Embarked_Q"] = (test["Embarked"] == "Q").astype(int)
test["Embarked_S"] = (test["Embarked"] == "S").astype(int)

# 6. Cabin 直接 drop
test = test.drop(columns=["Cabin"])

In [4]:
#### 特徵選擇 ####
features = [
    "Pclass",
    "Sex",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "FamilySize",
    "Embarked_Q",
    "Embarked_S",
]

X_test = test[features]

In [5]:
### 載入 Random Forest 模型
from sklearn.ensemble import RandomForestClassifier

# train_clean 也要切成 X / y
X_train = train_clean[features]
y_train = train_clean["Survived"]

rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight="balanced"
)
rf.fit(X_train, y_train)

# 對 test 預測
test_pred = rf.predict(X_test)

In [6]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
